In [22]:
from pyspark.sql import SparkSession
import tensorflow.compat.v2 as tf
import pandas as pd
import numpy as np
from PIL import Image
import os
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, udf
from pyspark.sql.functions import col, count, rand, collect_list, explode, struct, count, lit
from core import DetectorTF2
import glob
import xml.etree.ElementTree as ET
import pandas as pd


os.environ['PYSPARK_SUBMIT_ARGS'] = \
    '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.2,' \
    + 'org.apache.spark:spark-streaming-kafka-0-10_2.12:3.0.2,' \
    + 'org.apache.kafka:kafka-clients:3.1.0,org.apache.spark:spark-core_2.12:3.0.2,' \
    + 'org.apache.spark:spark-streaming_2.12:3.0.2,' \
    + 'org.apache.spark:spark-token-provider-kafka-0-10_2.12:3.0.2,' \
    + 'org.apache.hadoop:hadoop-core:1.2.1' \
    + ' pyspark-shell'


print("Tensorflow:", tf.__version__)

Tensorflow: 2.8.0


In [6]:
# Load SS, SC
ss  = SparkSession.builder.master('spark://spark-master:7077') \
                  .appName("test") \
                  .config("spark.executor.memory", "2g") \
                  .config("spark.driver.memory", "2g") \
                  .config("spark.executor.cores", "2") \
                  .config("spark.driver.cores", "2") \
                  .getOrCreate()
sc = ss.sparkContext
sc

<SparkContext master=spark://spark-master:7077 appName=test>

In [18]:
images = glob.glob("/usr/local/share_storages/data/image/id_cards/*JPG")
xmls = [item.replace(".JPG", ".xml") for item in images]

df = pd.DataFrame({'xml':xmls, 'image':images})

def get_para(xml):
    tree = ET.parse(xml)
    root = tree.getroot()
    for obj in root.findall('object'):
        bndbox = obj.find("bndbox")
        name = obj.find("name").text
        if name == 'paragraph':
            xmin = bndbox.find('xmin').text
            ymin = bndbox.find('ymin').text
            xmax = bndbox.find('xmax').text
            ymax = bndbox.find('ymax').text
            break
    return (int(xmin), int(ymin), int(xmax), int(ymax))


df['paragraph'] = df['xml'].apply(lambda x: get_para(x))
df

,xml,image,paragraph
0,/usr/local/share_storages/data/image/id_cards/...,/usr/local/share_storages/data/image/id_cards/...,"(86, 47, 1208, 743)"
1,/usr/local/share_storages/data/image/id_cards/...,/usr/local/share_storages/data/image/id_cards/...,"(1, 1, 1240, 775)"
2,/usr/local/share_storages/data/image/id_cards/...,/usr/local/share_storages/data/image/id_cards/...,"(1, 1, 1056, 676)"
3,/usr/local/share_storages/data/image/id_cards/...,/usr/local/share_storages/data/image/id_cards/...,"(44, 8, 1218, 728)"
4,/usr/local/share_storages/data/image/id_cards/...,/usr/local/share_storages/data/image/id_cards/...,"(4, 1, 1240, 768)"


In [19]:
df1 = ss.createDataFrame(df) 
df1.show()

+--------------------+--------------------+-------------------+
|                 xml|               image|          paragraph|
+--------------------+--------------------+-------------------+
|/usr/local/share_...|/usr/local/share_...|{86, 47, 1208, 743}|
|/usr/local/share_...|/usr/local/share_...|  {1, 1, 1240, 775}|
|/usr/local/share_...|/usr/local/share_...|  {1, 1, 1056, 676}|
|/usr/local/share_...|/usr/local/share_...| {44, 8, 1218, 728}|
|/usr/local/share_...|/usr/local/share_...|  {4, 1, 1240, 768}|
+--------------------+--------------------+-------------------+



In [26]:
MODEL_PATH = '/usr/local/share_storages/data/models/object_detection'
LABLE_PATH = '/usr/local/share_storages/data/models/object_detection/label_map.pbtxt'
NMS_THRESHOLD = 0.4
SCORE_THRESHOLD = 0.6
NUMBER_CLASSES = 80
MAX_CLASS_OUT = 100


# import os
# import tensorflow as tf
# from tensorflow.keras.preprocessing import image

pb_model_dir = "/usr/local/share_storages/data/models/object_detection/saved_model"
h5_model = "./mymodel.h5"

# Loading the Tensorflow Saved Model (PB)
model = tf.saved_model.load(pb_model_dir)
print(model.summary())

# Saving the Model in H5 Format
tf.keras.models.save_model(model, h5_model)

# Loading the H5 Saved Model
loaded_model_from_h5 = tf.keras.models.load_model(h5_model)
print(loaded_model_from_h5.summary())


AttributeError: '_UserObject' object has no attribute 'summary'

In [20]:
predictions = df1.withColumn("prediction", resnet50_udf(col("image")))

print(predictions.show())

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/site-packages/pyspark/serializers.py", line 437, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
  File "/usr/local/lib/python3.9/site-packages/pyspark/cloudpickle/cloudpickle_fast.py", line 73, in dumps
    cp.dump(obj)
  File "/usr/local/lib/python3.9/site-packages/pyspark/cloudpickle/cloudpickle_fast.py", line 563, in dump
    return Pickler.dump(self, obj)
TypeError: cannot pickle '_thread.RLock' object


PicklingError: Could not serialize object: TypeError: cannot pickle '_thread.RLock' object